In [32]:
import numpy as np 
import pandas as pd

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pylab import rcParams
rcParams['figure.figsize'] = (12,5)
%matplotlib inline

# time series related 
from pmdarima import auto_arima
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import mse,rmse
from statsmodels.tsa.statespace.varmax import VARMAX,VARMAXResults
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("Tatacoffee13_21.csv", index_col=0, parse_dates=True)


sp = pd.read_csv("varforecasted_28.csv", index_col=0, parse_dates=True)



In [33]:
df


,Open,High,Low,Close
Date,,,,
2013-01-01,1410.60,1427.90,1408.30,1415.10
2013-01-02,1421.00,1626.60,1416.15,1607.40
2013-01-03,1632.55,1673.90,1613.05,1626.20
2013-01-04,1627.75,1627.75,1574.60,1579.05
2013-01-07,1580.00,1639.50,1565.50,1595.65
...,...,...,...,...
2021-12-22,202.90,207.80,201.35,205.00
2021-12-23,206.00,206.85,202.05,202.95
2021-12-24,203.90,203.90,199.35,201.00


In [34]:
sp

,Close,High,Open,Low
0,0.651947,0.540348,0.637173,0.684598
1,0.646940,0.541544,0.631281,0.672035
2,0.640741,0.539336,0.626666,0.664086
3,0.634307,0.534927,0.621015,0.656887
4,0.628102,0.529992,0.615234,0.650382
5,0.622264,0.525105,0.609707,0.644445
6,0.616826,0.520457,0.604523,0.638983
7,0.611784,0.516108,0.599700,0.633945
8,0.607118,0.512068,0.595231,0.629292
9,0.602802,0.508326,0.591096,0.624993


In [35]:
from nsepy import get_history as gh
import datetime as dt

In [36]:
start = dt.datetime(2021,6,1)
end = dt.datetime(2022,2,11)
stk_data = gh(symbol='TATACOFFEE',start=start,end=end)


In [37]:
stk_data=stk_data[["Open","High","Low","Close"]]
#stk_data.to_csv("Tatacoffee13_21.csv")

In [38]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (177, 4)


In [39]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [40]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

142
X_train length: (142, 4)
X_test length: (35, 4)
y_train length: (142, 4)
y_test length: (35, 4)


In [41]:
import warnings
warnings.filterwarnings("ignore")

In [42]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [43]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [44]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [45]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -26.357935820450624
BIC:  -25.952907134806825

Order = 2
AIC:  -26.231032028762925
BIC:  -25.49868119118446

Order = 3
AIC:  -26.068132183966984
BIC:  -25.005477770755796

Order = 4
AIC:  -26.004242578592304
BIC:  -24.608257099388172

Order = 5
AIC:  -25.925452519076494
BIC:  -24.193061427657163

Order = 6
AIC:  -25.81298201903829
BIC:  -23.74106269717822

Order = 7
AIC:  -25.68548401422716
BIC:  -23.27086472773599

Order = 8
AIC:  -25.492416894076115
BIC:  -22.731875720105062

Order = 9
AIC:  -25.344561102070944
BIC:  -22.234824826741107

Order = 10
AIC:  -25.440206782579498
BIC:  -21.97794976795048



In [46]:
data1

,Open,High,Low,Close
0,0.140240,0.072381,0.073518,0.021463
1,0.049331,0.000000,0.064032,0.017488
2,0.096547,0.038730,0.120158,0.060413
3,0.110641,0.028571,0.090909,0.023052
4,0.089500,0.012698,0.079051,0.023052
...,...,...,...,...
172,0.613108,0.503492,0.667194,0.616057
173,0.591261,0.480635,0.633992,0.601749
174,0.633545,0.510476,0.687747,0.626391
175,0.633545,0.537778,0.695652,0.662162


In [48]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.118421,1,28
